<a href="https://colab.research.google.com/github/psychopatz/Colab-Torrent-Downloader-RELOADED/blob/main/Colab_Torrent_Downloader_Reloaded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://bit.ly/3xlijDv" target="_blank">
<img src="https://icons.iconarchive.com/icons/neiio/prime-dock-2/256/uTorrent-icon.png" height="100px" align="left"></a>

# <font size=6>Torrent Download <font color=#e53c1d>RE</font><font color=#16bf2d>LOA</font><font color=#158ff5>DEAD</font></font>



# A feature-rich **Torrent Download manager** powered by Google Collab.
**Features:**

<ul type- "disc">

<li>Built-in Torrent Search engine for fast secure search.</li>
<li>History and resume downloads support.</li>
<li>VPN-like features both downloading and searching that prevents DMCA Notice Since it technically uses Google's internet instead of yours.</li>
</ul>

Visit https://bit.ly/3xlijDv for more info.




In [1]:
#@title Initialize Downloader

#Drive Authentication
from google.colab import drive
drive.mount("/content/drive")


import os
import sys
import time
import shutil
import ipywidgets as widgets
from google.colab import files
from google.colab import output
from IPython.display import display
from IPython.display import clear_output

!python -m pip install --upgrade pip setuptools wheel
time.sleep(1)
clear_output()

!python -m pip install lbry-libtorrent
time.sleep(1)
clear_output()

!apt install python3-libtorrent
time.sleep(1)
clear_output()

!pip install bencodepy
time.sleep(1)
clear_output()

!pip install tpblite
time.sleep(1)
clear_output()



import base64
import hashlib
import bencodepy
import libtorrent as lt
from tpblite import TPB, CATEGORIES, ORDERS

time.sleep(1)
clear_output()
print('Importing Done..Now Authenticating GDrive ,')


#Environmental Variables
SaveDirectory = "/content/drive/My Drive/Torrent" #@param {type:"string"}
DeleteUnusedColabFiles = True #@param {type:"boolean"}

TorrentHistory = SaveDirectory + '/TorrentHistory/'
completed_path = TorrentHistory + 'Completed/'
deleted_path = TorrentHistory + 'Deleted/'
params = {"save_path": SaveDirectory}

for path in [TorrentHistory, SaveDirectory, completed_path]:
 if not os.path.exists(path):
	  os.makedirs(path)
   

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

color = {
   "PURPLE":'\033[95m',
   "CYAN":'\033[96m',
   "DARKCYAN":'\033[36m',
   "BLUE":'\033[94m',
   "GREEN":'\033[92m',
   "YELLOW":'\033[93m',
   "RED":'\033[91m',
   "BOLD":'\033[1m',
   "UNDERLINE":'\033[4m',
   "END":'\033[0m'
   
   }

######General Functions########
print ('Initialize Required Functions',end='')

def checkIfCpuRuntime():
    total, used, free = shutil.disk_usage("/")
    total = total // (2**30) 
    cpuRuntime = False
    if total < 108:
        cpuRuntime = True
    return cpuRuntime

def debloateColab():
    print("Deleting Unused Files:")
    print("*Old Python2.7")
    !rm -rf /usr/local/lib/python2.7
    print("*swift")
    !rm -rf /swift
    print("*tensorflow")
    !rm -rf /tensorflow-2.0.0
    
    if checkIfCpuRuntime:
        print("*Drivers")
        !rm -rf /usr/local/cuda-10.0
        !rm -rf /usr/local/cuda-10.1
        !rm -rf /opt/nvidia

    print("*Unused Packages")
    !rm -rf /usr/local/lib/python3.6/dist-packages/torch
    !rm -rf /usr/local/lib/python3.6/dist-packages/pystan
    !rm -rf /usr/local/lib/python3.6/dist-packages/spacy
    !rm -rf /content/sample_data

# extract magnet from .torrent
def torrent2Magnet(file) :
    metadata = bencodepy.decode_from_file(file)
    subj = metadata[b'info']
    hashcontents = bencodepy.encode(subj)
    digest = hashlib.sha1(hashcontents).digest()
    b32hash = base64.b32encode(digest).decode()
    
    return 'magnet:?'\
             + 'xt=urn:btih:' + b32hash\
             + '&dn=' + metadata[b'info'][b'name'].decode()\
             + '&tr=' + metadata[b'announce'].decode()\
             + '&xl=' + str(metadata[b'info'][b'piece length'])

def checkForDuplicates(data):
    found = False
    for index, download in enumerate(downloads[:]):
        if download.name() == data:
            found = True
            print("\n%sError%s: Torrent Already Existed on download Index: %d. File: %s" % (color["RED"],color["END"], index +1,download.name()))
            break

    return found

#Verify the magnet and download its metadata
def magnet2Torrent(magnet_link,saveToHistory):
    try:
        params = lt.parse_magnet_uri(magnet_link)
    except:
        print("%sError%s: Only valid Magnet link is accepted!" % (color["RED"],color["END"]))
        return False

    session = lt.session()
    handle = session.add_torrent(params)
    timer = 0
    print("Downloading Metadata", end='')
    while not handle.has_metadata(): 
        timer = timer + 1
        time.sleep(0.1)
        if timer == 100:
            print(".",end='')
        if timer == 250:
            print(".",end='')
        if timer == 500:
            print(".",end='')
        if timer == 750:
            print(".",end='')
        if timer == 850:
            print("It's slow due to %sLow Seeders%s" % (color["RED"],color["END"]))

        if timer >= 1000: #1 min estimation
            print("%sTimeout Error%s, can't download torrent metadata. Try again next time." % (color["RED"],color["END"]))
            return False 
    
    torrent_info = handle.get_torrent_info()
    if checkForDuplicates(torrent_info.name()):
        return False

# Create .torrent and save to the history
    if saveToHistory:
        torrent_file = lt.create_torrent(torrent_info)
        torrent_path = os.path.join(TorrentHistory, torrent_info.name() + ".torrent")
        
        with open(torrent_path, "wb") as f:
            f.write(lt.bencode(torrent_file.generate()))
        print ("\n %s is Added" % torrent_info.name())
    return True
      
#Show Queue
def showQueue():
  if downloads:
    print ("\n%sDownload Queue%s:" % (color["BOLD"],color["END"]))
    for index, download in enumerate(downloads[:]):
        print ("[%s%d] %s" %("0" if index < 10 else "",index + 1, download.name()))



#Transfer .torrent when completed 
def transferTorrent(torrentName):
    try:
      shutil.move(TorrentHistory + torrentName + '.torrent', completed_path + torrentName + '.torrent')
    except:
      return
      
print ('.',end='')



#add Magnet Downloads
def addViaMagnet():
    while True:
        showQueue()
        magnet_link = input("Enter Magnet Link Or Type Exit: ")
        clear_output()
        if magnet_link.lower() == "exit":
            break
        if magnet2Torrent(magnet_link,True):
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))

#add .torrent Downloads
def addViaTorrent():
    while True:
        showQueue()
        source = files.upload()
        filename = next(iter(source))
        magnet_link = torrent2Magnet(filename)
        if magnet2Torrent(magnet_link,True):
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))
        del source
        clear_output()


############Resume Downloads###########

def getUnfinished():
  paths = os.listdir(TorrentHistory)
  return [path for path in paths if path.endswith('.torrent')]
 
def addFileToQueue(data,checkMetadata):
    magnet_link = torrent2Magnet(TorrentHistory + data)
    if checkMetadata:
        print("%sChecking: %s %s" % (color['YELLOW'],data, color['END']))
        ifFound = magnet2Torrent(magnet_link,False)
        if ifFound:
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))
            print("\n%sAdded to download queue: %s %s" % (color['GREEN'],data,color['END']))
            return ifFound
    else:
        if not checkForDuplicates(data.split(".torrent")):
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))
            print("%sAdded to queue: %s %s" % (color['GREEN'],data,color['END']))
            return True
        else:
            return False
        
def removeFromList(Items,data):
    Items.remove(data) 

def addFromList(Items,data):
    Items.append(data)

def deleteTorrent(torrentName):
    try:
        shutil.move(TorrentHistory + torrentName,deleted_path + torrentName)
    except:
        print(".", end="")

def exceededRecomendedLimit():
    notDone = True
    lastCheck = input("Choice: ")
    while lastCheck.lower() not in ("y", "n"):
          print("%sInvalid%s Input, valid input: Y/N. " % (color['RED'],color['END']))
          lastCheck = input("Choice: ")
            
          if lastCheck.lower() == "y":
              notDone = False
    return notDone

def checkIfNotExceed(data,queue,startingIndex):
    exceeded = False
    if data < startingIndex or data > len(queue) - 1:
        print("%sInvalid input%s. Must choose only between %d to %d" % (color['RED'],color['END'],startingIndex + 1,len(queue)) )
        input("Press Enter to continue...")
        exceeded = True
    
    return exceeded

def deleteTorrentMenu():
    startingIndex = 0
    amountToShow = 15
    notDone = True
    while notDone:
        clear_output()
        torrentFiles = getUnfinished()
        print("Enter torrent to be %sdeleted%s or \"Exit\":\n" % (color['RED'],color['END']))
        displayData(torrentFiles,startingIndex,amountToShow)
        userInput = input("Choice: ")
        if userInput.lower() == "exit":
            notDone = False
            continue

        try:
            userInput = int(userInput)
            userInput = userInput - 1
        except ValueError:
            print("%sError: Invalid input %s" % (color['RED'],color['END']))
            input("Press Enter to continue...")
            continue
        
        if checkIfNotExceed(userInput,torrentFiles,startingIndex):
            continue

        deleteTorrent(torrentFiles[userInput])

              

def displayData(Items,startingIndex,amountToShow):
    index = startingIndex
    currentIndex = 0
    if len(Items):
        for data in Items:
            print("[%s%d].%s" % (0 if index < 9 else '',index+1, data))
            index = index + 1
    
    else:
        print("Currently Empty....")
              

def fileManager():
    currentQueue = getUnfinished()
    selectedDownloadQueue = []
    recommendedLimit = 6
    startingIndex = 0
    notDone = True
    amountToShow = 15

    while notDone:
        clear_output()
        print("%s" % ("Type the index you want to add from the queue." if len(selectedDownloadQueue) < recommendedLimit else "You have too many items on the queue, consider removing some."))
        print("\"All\" to select all, \"remove\" to remove from queue, or type \"Exit\".\n")
        print("%sUnfinished Downloads: " % (color['DARKCYAN']))
        displayData(currentQueue,startingIndex,amountToShow)
        print("\n%sSelected Download Queue:" % (color['GREEN']))
        displayData(selectedDownloadQueue,startingIndex,amountToShow)
        print(color['END'])

        userInput = input("choice: ")

        if userInput.lower() == "exit":
            lastCheck = "y"
            if not len(selectedDownloadQueue):
                lastCheck = input("The download queue is currently empty, are you sure to exit? Y/N.\nChoice: ")
                while lastCheck.lower() not in ("y","n"):
                     print("%sInvalid%s Input, valid input: Y/N" % (color['RED'],color['END']))
                     lastCheck = input("Choice: ")

            if lastCheck.lower() == "y":
                print("Selection Done...")         
                notDone = False
            continue

        elif userInput.lower() == "all":
            if len(currentQueue)== 0:
                print("no more files to add.")
                input("Press Enter to continue...")

            selectedDownloadQueue = selectedDownloadQueue + currentQueue
            currentQueue.clear()
            continue
        
        if userInput.lower() == "clear":
            if len(selectedDownloadQueue)== 0:
                print("The queue is empty, theres nothing to clear.")
                input("Press Enter to continue...")

            currentQueue = currentQueue + selectedDownloadQueue
            selectedDownloadQueue.clear()
            continue

        elif userInput.lower() == "remove":
            clear_output()
            if not len(selectedDownloadQueue):
                print("The queue is currently empty, theres nothing to remove.")
                input("Press Enter to continue...")
                continue

            print("%sSelect Data to remove from queue:%s" % (color["RED"],color["DARKCYAN"]))
            displayData(selectedDownloadQueue,startingIndex,amountToShow)
            print(color["END"])

            running = True
            while running:
                try:
                    userInput = int(input("Choice: "))
                    userInput = userInput - 1
                except ValueError:
                    print("%sError: Invalid input %s" % (color['RED'],color['END']))
                    input("Press Enter to continue...")
                    continue
                
                if checkIfNotExceed(userInput,selectedDownloadQueue,startingIndex):
                    continue
              
                running = False
           

            addFromList(currentQueue,selectedDownloadQueue[userInput])
            removeFromList(selectedDownloadQueue,selectedDownloadQueue[userInput])
            continue

        try:
            userInput = int(userInput)
            userInput = userInput - 1

        except ValueError:  
            print("%sError: Invalid input %s" % (color['RED'],color['END']))
            input("Press Enter to continue...")
            continue

        if checkIfNotExceed(userInput,currentQueue,startingIndex):
            continue

        addFromList(selectedDownloadQueue,currentQueue[userInput])
        removeFromList(currentQueue,currentQueue[userInput])

        if len(selectedDownloadQueue) ==  recommendedLimit:
            clear_output()
            print("\n%sSelected Download Queue:" % (color['GREEN']))
            displayData(selectedDownloadQueue,startingIndex,amountToShow)
            print(color['END'])
            print("\nThe current selected queue now exceeds the recommended limit which is %d. Do you want to start downloading now? Y/N." % (recommendedLimit))
                    
            notDone = exceededRecomendedLimit()

    return selectedDownloadQueue

def menuResumeUnfinished():
       
    if len(getUnfinished()) > 0: 
        downloadQueue = []
        recommendedLimit = 6

        #unimplemented Feature
        startingIndex = 0 
        amountToShow = 15

        print("Menu:\n[1]Resume Unfinished torrents.(Default)\n[2]Delete Selected Torrents\n[3]Clear Recent Torrents")
        userInput =input("choice: ")

        if not userInput:
          userInput = 1
          
        try:
            userInput = int(userInput)
        except ValueError:
            print("%sError: Invalid input %s" % (color['RED'],color['END']))

        while userInput not in (1,2,3):
            print("%sError: Invalid input %s" % (color['RED'],color['END']))
            userInput = int(input("Choice: "))
        
        if userInput == 1:
            downloadQueue = fileManager()
        
        elif userInput == 2:
            deleteTorrentMenu()
            downloadQueue = fileManager()

        elif userInput == 3:
            print("These data will be moved in \"Deleted\" folder")
            for data in downloadQueue:
                print(data)
                deleteTorrent(data)
        
        if len(downloadQueue) != 0:
            print("Do you want to verify %s for seeders? Y/N  %s(May take longer) %s" % ("these torrents" if len(getUnfinished()) > 1 else "this torrent",color['YELLOW'],color['END']))
            checkMetadata = input("Choice: ")
            while checkMetadata.lower() not in ("y","n"):
                print("%sError: Invalid input %s" % (color['RED'],color['END']))
                checkMetadata = input("Choice: ")

            clear_output()
            
            if checkMetadata == "y":
                checkMetadata = True
            else:
                checkMetadata = False
            
            invalid = 0
            for data in downloadQueue:
                if not addFileToQueue(data,checkMetadata):
                    invalid = invalid + 1
            
            print("%sImport complete. Errors found: %d %s" % (color['CYAN'],invalid,color['END']))
            input("Press Enter to continue...")
            
            clear_output()
            showQueue()
            input("Press Enter to continue...")   
    else:
        print("There's currently no unfinished download at the moment. Now go get some..")
        input("Press Enter to continue...")

############Torrent Searcher###########


############Starting Procedures########
print(".")
if DeleteUnusedColabFiles:
    debloateColab()


print('Importing Done..Mounting GDrive Done,')
print('Initializiation Complete,')
time.sleep(1)
print('Now you can run the next cell')



Importing Done..Now Authenticating GDrive ,
Initialize Required Functions..
Deleting Unused Files:
*Old Python2.7
*swift
*tensorflow
*Drivers
*Unused Packages
Importing Done..Mounting GDrive Done,
Initializiation Complete,
Now you can run the next cell


In [ ]:
#@title Use the Program
#<ul type- "disc">
#<li>1</li>
#<li>History and resume downloads support.</li>
#<li>fd</li>
#</ul>


###Banners

def bannerTorrentDownloader():
    print("%s     _____                         _    %s______                   %s_                  _           " % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s    |_..._|                       | |   %s|  _  \                  %s| |                | |          " % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | ___  _ __ _ __ ___ _ __ | |_  %s| | | |_____      ___ _ _%s| |  ___   __ _  __| | ___ _ __ " % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | |/ _ \| '__| '__/ _ | '_ \| __| %s| | | / _ \ \ /\ / | '_ \| |/ _ \%s / _` |/ _` |/ _ | '__|" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | (_) | |  | | |  __| | | | |_  %s| |/ | (_) \ V  V /| | | | | (_) | (_| | (_| |  __| |   " % (color["GREEN"],color["BLUE"]))
    print("%s      \_/\___/|_|  |_|  \___|_| |_|\__| %s|___/ \___/ \_/\_/ |_| |_|_|\___/ \__,_|\__,_|\___|_|   " % (color["GREEN"],color["BLUE"]))
    print("%s                                                                                                   " % (color["END"]))

###Beta Functions

def bannerTorrentSearcher():
    print("%s     _____ %s                         _  %s  _____                     _       " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s    |_   _|%s                        | | %s /  ___|                   | |      " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | |%s  ___  _ __ _ __ ___ _ __ | |_%s \ `--.  ___  __ _ _ __ ___| |__    " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | %s|/ _ \| '__| '__/ _ | '_ \| __|%s  `--. \/ _ \/ _` | '%s__/ __| '_ \ " % (color["PURPLE"],color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | (_) | |  | | |  __| | | | |_ %s /\__/ |  __| (_| | |%s | (__| | | |" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      \_/\___/|_|  |_|  \___|_| |_|\__|%s \____/ \___|\__,_|_|%s  \___|_| |_|" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s                                                                             " % (color["END"]))
    
def printContents(torrents):
    index=0
    for torrent in torrents:
        index = index + 1
        #print("[%d]%s , %sSeeds: %s %-5s%s, %sSize: %s%s, VIP: %s, Trusted: %s" % (index, ' ' if index < 10 else '',color["GREEN"], torrent.seeds,color["END"], torrent.title, color["PURPLE"], torrent.filesize,color["END"] , torrent.is_vip, torrent.is_trusted))
        print("[%s%s]" % ('0' if index < 10 else '', index), end='')
        #print(color["DARKCYAN"],"",end='')
        print("%s%s" % ( color["GREEN"] if torrent.is_vip else color["PURPLE"] if torrent.is_trusted else color["RED"],'🕱'),end='')
        #print(color["DARKCYAN"],"",end='')
        print(color["UNDERLINE"],end='')
        print(color["GREEN"],end='')
        print("[Seed:%s]" % (torrent.seeds), end='')
        print(color["DARKCYAN"],end='')
        print("%s " % (torrent.title), end='')
        print(color["CYAN"],end='')
        print("[Size:%s]" % (torrent.filesize),end='')
        print(color["END"])

def searchNew():
    bannerTorrentSearcher()
    keyword = ""

    while True:
        try:
            keyword = input("Search Torrent: ")
        except EOFError:
            print("%sError%s: You must put a value in order to search!" % (color["RED"],color["END"]))
            continue

        return keyword



def getChooseInput(maxRange):

    userInput = input("Choose: ")

    
    #[0]quit, [-1]error, [-2]next, [-3]back, [4]newSearch
    if userInput.isalpha():
        if userInput.lower() == 's' or userInput.lower() == "search": #newSearch
            return -4
        elif userInput.lower() == 'x' or userInput.lower() == "exit": #quit
            return 0
        elif userInput.lower() == 'n' or userInput.lower() == "next": #next
            return -2
        elif userInput.lower() == 'b' or userInput.lower() == "back": #back
            return -3
        else:
            print("\n%s Invalid Command%s" % (color["RED"],color["END"]))
            input("Press Enter to continue...")
            return -1
    try:
        userInput = int(userInput)
    except ValueError:
        print("%sError%s: You must input a value" %(color["RED"],color["END"]))
        input("Press Enter to continue...")
        return -1

    if userInput <= 0 or userInput > maxRange:
        print("%s Error %s: Invalid Input: Must be between 0-%d" %(color["RED"],color["END"],maxRange))
        input("Press Enter to continue...")
        return -1

    return userInput



def detailedItemView(torrent):
    print("\nSelected torrent:\n")
    print("Name: " + torrent.title)
    print("Category: " + torrent.category)
    print("Size: " + torrent.filesize)
    print("Seeder: %d\n" % (torrent.seeds))
    print("Date Uploaded: " + torrent.upload_date)
    print("Uploader: " + torrent.uploader)
    print("Reputation: %s" %("VIP" if torrent.is_vip else "Trusted" if torrent.is_trusted else "Unverified"))
    #print(CATEGORIES.printOptions())
    #print(ORDERS.printOptions())
    print("URL: " + torrent.url)

def askToDownload(magnetlink):
    option = input("\nDownload this File? Y/N: ")
    while option.lower() not in ("y","n"):
        option = input("%s Invalid Syntax %s, Download this File? Y/N: " % (color["RED"],color["END"]))
    if option == "y":
        if magnet2Torrent(magnetlink,True):
            downloads.append(lt.add_magnet_uri(ses, magnetlink, params))
            return True
    input("Press Enter to continue...")
    return False

def torrentSearcher():
    Search = searchNew()
    t = TPB()
    page = 1

    while True:
        print("Please Wait...")
        torrents = t.search(Search,page,ORDERS.SEEDERS.DES)
        clear_output()
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        print("\t%sSearch Results of: %s" % (color["BOLD"],Search))
        print("\tPage: %s\t Current Items: %d\t Category: All\t Sort: Seeders/Descending %s" % (page,len(torrents), color["END"]))
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        printContents(torrents)
        print("\n%sOptions: [S]earch, [N]ext, [B]ack, E[x]it:%s\n" % (color["BOLD"],color["END"]))
        userInput = getChooseInput(len(torrents))
        #[0]quit, [-1]error, [-2]next, [-3]back, [-4]newSearch
        
        if userInput == 0:
            print("Now Exiting Search Engine...")
            break
        
        elif userInput == -2:
            if len(torrents) == 30:
                page = page + 1
            else:
                print("%sError:%s This is the last page.")
                input("Press Enter to continue...")
            continue

        elif userInput == -3:
            if page - 1 > 0:
                page = page - 1
                continue
            else:
                print("Error, you're in the starting page already. ")
                input("Press Enter to continue...")
                continue
        elif userInput == -4:
            Search = searchNew()
            page = 1
            continue
        elif userInput > 0:
            torrent = torrents[userInput-1]
            detailedItemView(torrent)
            if askToDownload(torrent.magnetlink):
                print ("\nImport Complete") 
                showQueue()
                input("Press Enter to continue...")
                continue


def completeDialogue(name):
    print("%s \u272F%s %s %sComplete%s" % (color["YELLOW"],color["END"],name, color["GREEN"],color["END"]))



#Main
params = {"save_path": SaveDirectory}
Menu = "Resume Downloads" #@param ["Search Torrents","Add Magnet", "Add torrent", "Resume Downloads"]


if Menu == "Add Magnet" :
    addViaMagnet()
elif Menu == "Add .torrent" :
    try: 
        addViaTorrent()
    except:
      print ("Selection Canceled..")
elif Menu == "Resume Downloads" :
    menuResumeUnfinished()
elif Menu == "Search Torrents" :
    torrentSearcher()


#elif Add == "resume downloads" planned feature



time.sleep(2)
clear_output()
#print (color["UNDERLINE"],color["BOLD"],"Now Downloading:",color["END"])
bannerTorrentDownloader()
state_str = [
    "Queued:",
    "&#10061 Checking:",
    "\u267B Downloading Metadata:",
    "🡇 Downloading:",
    "Finished:",
    "Seeding",
    "\u221E Allocating:",
    "Checking fastresume:",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    state_str[s.state],
                    download.name(),
                    ",&#9684Seeders: ",str(s.num_seeds),
                    ",⇅Speed: ", str(s.download_rate / 1000),"kB/s",
                    
                ]
            )
            bar.value = s.progress * 100  
        else:
            next_shift -= 1
            completeDialogue(download.name())
            transferTorrent(download.name())
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() 
            download_bars.remove(bar)
            
    time.sleep(1)

print("\n\u2705Program Terminated, No more pending downloads!")


     _____                         _    ______                   _                  _           
    |_..._|                       | |   |  _  \                  | |                | |          
      | | ___  _ __ _ __ ___ _ __ | |_  | | | |_____      ___ _ _| |  ___   __ _  __| | ___ _ __ 
      | |/ _ \| '__| '__/ _ | '_ \| __| | | | / _ \ \ /\ / | '_ \| |/ _ \ / _` |/ _` |/ _ | '__|
      | | (_) | |  | | |  __| | | | |_  | |/ | (_) \ V  V /| | | | | (_) | (_| | (_| |  __| |   
      \_/\___/|_|  |_|  \___|_| |_|\__| |___/ \___/ \_/\_/ |_| |_|_|\___/ \__,_|\__,_|\___|_|   
                                                                                                   


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

 ✯ 50 C Programming Source Codes Explained Complete
 ✯ [FreeTutorials.Us] Udemy - Learn Python & Ethical Hacking From Scratch Complete


#Code Sandbox
<h5> Here Lies the unfinished codes. For Devs only.</h5>

In [ ]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/commons/e/e3/Animhorse.gif')

In [ ]:
def debloateColab():
    print("Deleting Unused Files:")
    print("*Python2.7")
    !rm -rf /usr/local/lib/python2.7
    print("*swift")
    !rm -rf /swift
    print("*Cuda-10.0")
    !rm -rf /usr/local/cuda-10.0
    print("*Cuda-10.1")
    !rm -rf /usr/local/cuda-10.1
    print("*tensorflow-2.0.0")
    !rm -rf /tensorflow-2.0.0
    print("*Torch")
    !rm -rf /usr/local/lib/python3.6/dist-packages/torch
    print("*Nvidia")
    !rm -rf /opt/nvidia
    print("*Pystan")
    !rm -rf /usr/local/lib/python3.6/dist-packages/pystan
    print("*Spacy")
    !rm -rf /usr/local/lib/python3.6/dist-packages/spacy

debloateColab()

Deleting Unused Files:
*Python2.7
*swift
*Cuda-10.1
*Cuda-10.0
*tensorflow-2.0.0
*Torch
*Nvidia
*Pystan
*Spacy


In [ ]:
category = dict()
category["All"]= CATEGORIES.ALL
category["Audio"]= CATEGORIES.AUDIO.ALL
category["Audio/Music"]= CATEGORIES.AUDIO.MUSIC
category["Audio/Audio Books"]= CATEGORIES.AUDIO.AUDIO_BOOKS
category["Audio/Sound Clips"]= CATEGORIES.AUDIO.SOUND_CLIPS
category["Audio/FLAC"]= CATEGORIES.AUDIO.FLAC
category["Audio/Other"]= CATEGORIES.AUDIO.OTHER
category["Videos"]= CATEGORIES.VIDEO.ALL
category["Videos/Movies"]= CATEGORIES.VIDEO.MOVIES
category["Videos/Movies DVDR"]= CATEGORIES.VIDEO.MOVIES_DVDR
category["Videos/Music Videos"]= CATEGORIES.VIDEO.MUSIC_VIDEOS
category["Videos/Movie Clips"]= CATEGORIES.VIDEO.MOVIE_CLIPS
category["Videos/TV Shows"]= CATEGORIES.VIDEO.TV_SHOWS
category["Videos/Video Handheld"]= CATEGORIES.VIDEO.HANDHELD
category["Videos/HD Movies"]= CATEGORIES.VIDEO.HD_MOVIES
category["Videos/HD TV Shows"]= CATEGORIES.VIDEO.HD_TV_SHOWS
category["Videos/3D"]= CATEGORIES.VIDEO.THREE_DIMENSIONS
category["Videos/Others"]= CATEGORIES.VIDEO.OTHER
category["Apps"]= CATEGORIES.APPLICATIONS.ALL
category["Apps/Windows"]= CATEGORIES.APPLICATIONS.WINDOWS
category["Apps/Mac"]= CATEGORIES.APPLICATIONS.MAC
category["Apps/Unix"]= CATEGORIES.APPLICATIONS.UNIX
category["Apps/Handheld"]= CATEGORIES.APPLICATIONS.HANDHELD
category["Apps/IOS"]= CATEGORIES.APPLICATIONS.IOS
category["Apps/Android"]= CATEGORIES.APPLICATIONS.ANDROID
category["Apps/Others"]= CATEGORIES.APPLICATIONS.OTHER
category["Games"]= CATEGORIES.GAMES.ALL
category["Games/PC"]= CATEGORIES.GAMES.PC
category["Games/Mac"]= CATEGORIES.GAMES.MAC
category["Games/Psx"]= CATEGORIES.GAMES.PSX
category["Games/Xbox 360"]= CATEGORIES.GAMES.XBOX360
category["Games/WII"]= CATEGORIES.GAMES.WII
category["Games/Handheld"]= CATEGORIES.GAMES.HANDHELD
category["Games/IOS"]= CATEGORIES.GAMES.IOS
category["Games/Android"]= CATEGORIES.GAMES.ANDROID
category["Games/Others"]= CATEGORIES.GAMES.OTHER
category["Porn"]= CATEGORIES.PORN.ALL
category["Porn/Movies"]= CATEGORIES.PORN.MOVIES
category["Porn/Movies DVDR"]= CATEGORIES.PORN.MOVIES_DVDR
category["Porn/Pictures"]= CATEGORIES.PORN.PICTURES
category["Porn/Games"]= CATEGORIES.PORN.GAMES
category["Porn/HD Movies"]= CATEGORIES.PORN.HD_MOVIES
category["Porn/Movie Clips"]= CATEGORIES.PORN.MOVIE_CLIPS
category["Porn/Others"]= CATEGORIES.PORN.OTHER
category["Other/Ebooks"]= CATEGORIES.OTHER.EBOOKS
category["Other/Comics"]= CATEGORIES.OTHER.COMICS
category["Other/Pictures"]= CATEGORIES.OTHER.PICTURES
category["Other/Covers"]= CATEGORIES.OTHER.COVERS
category["Other/Physibles"]= CATEGORIES.OTHER.PHYSIBLES
category["Other/Others"]= CATEGORIES.OTHER.OTHER

print (category)
print (CATEGORIES.AUDIO.ALL)

{'All': 0, 'Audio': 100, 'Audio/Music': 101, 'Audio/Audio Books': 102, 'Audio/Sound Clips': 103, 'Audio/FLAC': 104, 'Audio/Other': 199, 'Videos': 200, 'Videos/Movies': 201, 'Videos/Movies DVDR': 202, 'Videos/Music Videos': 203, 'Videos/Movie Clips': 204, 'Videos/TV Shows': 205, 'Videos/Video Handheld': 206, 'Videos/HD Movies': 207, 'Videos/HD TV Shows': 208, 'Videos/3D': 209, 'Videos/Others': 299, 'Apps': 300, 'Apps/Windows': 301, 'Apps/Mac': 302, 'Apps/Unix': 303, 'Apps/Handheld': 304, 'Apps/IOS': 305, 'Apps/Android': 306, 'Apps/Others': 399, 'Games': 400, 'Games/PC': 401, 'Games/Mac': 402, 'Games/Psx': 403, 'Games/Xbox 360': 404, 'Games/WII': 405, 'Games/Handheld': 406, 'Games/IOS': 407, 'Games/Android': 408, 'Games/Others': 499, 'Porn': 500, 'Porn/Movies': 501, 'Porn/Movies DVDR': 502, 'Porn/Pictures': 503, 'Porn/Games': 504, 'Porn/HD Movies': 505, 'Porn/Movie Clips': 506, 'Porn/Others': 599, 'Other/Ebooks': 601, 'Other/Comics': 602, 'Other/Pictures': 603, 'Other/Covers': 604, 'Othe

In [ ]:

def bannerTorrentSearcher():
    print("%s     _____ %s                         _   %s  _____                     _       " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s    |_   _|%s                        | |  %s /  ___|                   | |      " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | |%s  ___  _ __ _ __ ___ _ __ | |_ %s \ `--.  ___  __ _ _ __ ___| |__    " % (color["PURPLE"],color["GREEN"],color["BLUE"]))
    print("%s      | %s|/ _ \| '__| '__/ _ | '_ \| __|%s  `--. \/ _ \/ _` | '%s__/ __| '_ \ " % (color["PURPLE"],color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      | | (_) | |  | | |  __| | | | |_ %s /\__/ |  __| (_| | |%s | (__| | | |" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s      \_/\___/|_|  |_|  \___|_| |_|\__|%s \____/ \___|\__,_|_|%s  \___|_| |_|" % (color["GREEN"],color["BLUE"],color["RED"]))
    print("%s                                                                             " % (color["END"]))
    
def printContents(torrents):
    index=0
    for torrent in torrents:
        index = index + 1
        #print("[%d]%s , %sSeeds: %s %-5s%s, %sSize: %s%s, VIP: %s, Trusted: %s" % (index, ' ' if index < 10 else '',color["GREEN"], torrent.seeds,color["END"], torrent.title, color["PURPLE"], torrent.filesize,color["END"] , torrent.is_vip, torrent.is_trusted))
        print("[%s%s]" % ('0' if index < 10 else '', index), end='')
        #print(color["DARKCYAN"],"",end='')
        print("%s%s" % ( color["GREEN"] if torrent.is_vip else color["PURPLE"] if torrent.is_trusted else color["RED"],'🕱'),end='')
        #print(color["DARKCYAN"],"",end='')
        print(color["UNDERLINE"],end='')
        print(color["GREEN"],end='')
        print("[Seed:%s]" % (torrent.seeds), end='')
        print(color["DARKCYAN"],end='')
        print("%s " % (torrent.title), end='')
        print(color["CYAN"],end='')
        print("[Size:%s]" % (torrent.filesize),end='')
        print(color["END"])

def searchNew():
    bannerTorrentSearcher()
    keyword = ""

    while True:
        try:
            keyword = input("Search Torrent: ")
        except EOFError:
            print("%sError%s: You must put a value in order to search!" % (color["RED"],color["END"]))
            continue

        return keyword



def getChooseInput(maxRange):

    userInput = input("Choose: ")

    
    #[0]quit, [-1]error, [-2]next, [-3]back, [4]newSearch
    if userInput.isalpha():
        if userInput.lower() == 's' or userInput.lower() == "search": #newSearch
            return -4
        elif userInput.lower() == 'x' or userInput.lower() == "exit": #quit
            return 0
        elif userInput.lower() == 'n' or userInput.lower() == "next": #next
            return -2
        elif userInput.lower() == 'b' or userInput.lower() == "back": #back
            return -3
        else:
            print("\n%s Invalid Command%s" % (color["RED"],color["END"]))
            input("Press Enter to continue...")
            return -1
    try:
        userInput = int(userInput)
    except ValueError:
        print("%sError%s: You must input a value" %(color["RED"],color["END"]))
        input("Press Enter to continue...")
        return -1

    if userInput <= 0 or userInput > maxRange:
        print("%sError%s: Invalid Input: Must be between 0-%d" %(color["RED"],color["END"],maxRange))
        input("Press Enter to continue...")
        return -1

    return userInput



def detailedItemView(torrent):
    print("\nSelected torrent:\n")
    print("Name: " + torrent.title)
    print("Category: " + torrent.category)
    print("Size: " + torrent.filesize)
    print("Seeder: %d\n" % (torrent.seeds))
    print("Date Uploaded: " + torrent.upload_date)
    print("Uploader: " + torrent.uploader)
    print("Reputation: %s" %("VIP" if torrent.is_vip else "Trusted" if torrent.is_trusted else "Unverified"))
    #print(CATEGORIES.printOptions())
    #print(ORDERS.printOptions())
    print("URL: " + torrent.url)

def askToDownload(magnetlink):
    option = input("\nDownload File? Y/N: ")
    while option.lower() not in ("y","n"):
        option = input("%s Invalid Syntax%s, Download File? Y/N: " % (color["RED"],color["END"]))
    if option == "y":
        if magnet2Torrent(magnetlink,True):
            downloads.append(lt.add_magnet_uri(ses, magnetlink, params))
            return True
    input("Press Enter to continue...")
    return False

def torrentSearcher():
    Search = searchNew()
    t = TPB()
    page = 1
    while True:
        print("Please Wait...")
        #torrents = t.search(Search,page,ORDERS.SEEDERS.DES)

        torrents = t.top(CATEGORIES.ALL)

        clear_output()
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        print("\t%sSearch Results of: %s" % (color["BOLD"],Search))
        print("\tPage: %s\t Current Items: %d\t Category: All\t Sort: Seeders/Descending %s" % (page,len(torrents), color["END"]))
        print("-=-=-=-=-=-=-=-=-=---=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=--=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        printContents(torrents)
        print("\n%sOptions: [S]earch, [N]ext, [B]ack, E[x]it:%s\n" % (color["BOLD"],color["END"]))
        userInput = getChooseInput(len(torrents))
        #[0]quit, [-1]error, [-2]next, [-3]back, [-4]newSearch
        
        if userInput == 0:
            print("Now Exiting Search Engine...")
            break
        
        elif userInput == -2:
            if len(torrents) == 30:
                page = page + 1
            else:
                print("Error: This is the last page.")
                input("Press Enter to continue...")
            continue

        elif userInput == -3:
            if page - 1 > 0:
                page = page - 1
                continue
            else:
                print("Error, you're in the starting page already. ")
                input("Press Enter to continue...")
                continue
        elif userInput == -4:
            Search = searchNew()
            page = 1
            continue
        elif userInput > 0:
            torrent = torrents[userInput-1]
            detailedItemView(torrent)
            if askToDownload(torrent.magnetlink):
                print ("\nImport Complete") 
                showQueue()
                input("Press Enter to continue...")
                continue
        else :
            print("%sError%s, Adding unsuccessful" % (color["RED"],color["END"]))
            input("Press Enter to continue...")


torrentSearcher()


In [ ]:
print(CATEGORIES.printOptions())

ALL
AUDIO.ALL
AUDIO.MUSIC
AUDIO.AUDIO_BOOKS
AUDIO.SOUND_CLIPS
AUDIO.FLAC
AUDIO.OTHER
VIDEO.ALL
VIDEO.MOVIES
VIDEO.MOVIES_DVDR
VIDEO.MUSIC_VIDEOS
VIDEO.MOVIE_CLIPS
VIDEO.TV_SHOWS
VIDEO.HANDHELD
VIDEO.HD_MOVIES
VIDEO.HD_TV_SHOWS
VIDEO.THREE_DIMENSIONS
VIDEO.OTHER
APPLICATIONS.ALL
APPLICATIONS.WINDOWS
APPLICATIONS.MAC
APPLICATIONS.UNIX
APPLICATIONS.HANDHELD
APPLICATIONS.IOS
APPLICATIONS.ANDROID
APPLICATIONS.OTHER
GAMES.ALL
GAMES.PC
GAMES.MAC
GAMES.PSX
GAMES.XBOX360
GAMES.WII
GAMES.HANDHELD
GAMES.IOS
GAMES.ANDROID
GAMES.OTHER
PORN.ALL
PORN.MOVIES
PORN.MOVIES_DVDR
PORN.PICTURES
PORN.GAMES
PORN.HD_MOVIES
PORN.MOVIE_CLIPS
PORN.OTHER
OTHER.EBOOKS
OTHER.COMICS
OTHER.PICTURES
OTHER.COVERS
OTHER.PHYSIBLES
OTHER.OTHER
None


In [ ]:
print(ORDERS.printOptions())

NAME.DES
NAME.ASC
UPLOADED.DES
UPLOADED.ASC
SIZE.DES
SIZE.ASC
SEEDERS.DES
SEEDERS.ASC
LEECHERS.DES
LEECHERS.ASC
UPLOADER.DES
UPLOADER.ASC
TYPE.DES
TYPE.ASC
None
